In [ ]:
!pip install tqdm

In [ ]:
import os
import glob
import numpy as np
import nibabel as nib
from tqdm import tqdm

In [ ]:
mask_files = glob.glob('/ngochuynh/f/Dataset/ADNI/atlas/mask/combined_mask/*')
image_path = '/ngochuynh/f/Dataset/ADNI/ADNI_renamed'
dest_dir   = '/ngochuynh/f/Dataset/ADNI/ADNI_regions' 
image_type = 'MRI'
image_files = glob.glob(f'{image_path}/*/preprocessed/{image_type}/transformed*')

In [ ]:
def extract_brain_regions(img, mask_paths, output_dir):
    for mpath in mask_paths:
        mname = os.path.splitext(os.path.basename(mpath))[0]
        region_mask = nib.load(mpath)
        region_data = img.get_fdata() * region_mask.get_fdata()
        region_img = nib.Nifti1Image(region_data, img.affine)
        # Get the bounding box of the region
        x, y, z = np.where(region_data != 0)
        min_x, max_x = min(x), max(x)
        min_y, max_y = min(y), max(y)
        min_z, max_z = min(z), max(z)
        # Crop the region using the bounding box
        cropped_region_data = region_data[min_x:max_x+1, min_y:max_y+1, min_z:max_z+1]
        cropped_region_img = nib.Nifti1Image(cropped_region_data, region_img.affine)
        nib.save(cropped_region_img, os.path.join(output_dir, f'masked_{mname[5:]}.nii'))

In [ ]:
total_rows   = len(image_files)
progress_bar = tqdm(total=total_rows, desc="Processing Files")

for img_path in image_files:
    basename = os.path.splitext(os.path.basename(img_path))[0]
    ptid = basename[12:22]
    output_dir = os.path.join(dest_dir, ptid, image_type, basename)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
        img = nib.load(img_path)
        extract_brain_regions(img, mask_files, output_dir)
    
    progress_bar.update(1)
progress_bar.close()

In [ ]:
output_dir